In [4]:
import maup # mggg's library for proration, see documentation here: https://github.com/mggg/maup
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files

# VEST Florida Validation

In [46]:
vest_fl_16 = gp.read_file("./raw-from-source/VEST/fl_2016/fl_2016.shp")

## Documentation

### Races

G16PRERTru - Donald J. Trump (Republican Party)  
G16PREDCli - Hillary Clinton (Democratic Party)  
G16PRELJoh - Gary Johnson (Libertarian Party)  
G16PRECCas - Darrell L. Castle (Constitution Party)  
G16PREGSte - Jill Stein (Green Party)  
G16PREIDeL - Roque De La Fuente (Reform Party)  
G16PREOth - Write-in Votes  

G16USSRRub - Marco Rubio (Republican Party)  
G16USSDMur - Patrick Murphy (Democratic Party)  
G16USSLSta - Paul Stanton (Libertarian Party)  
G16USSOth - Independent and Write-in Votes  

### Election Source

Election results from Florida Division of Elections (http://dos.myflorida.com/elections/data-statistics/elections-data/precinct-level-election-results/)  

### Precinct Source

The starting point was the Orlando Sentinel's precinct map for the 2016 presidential primary (http://interactive.orlandosentinel.com/elections/2016/presidential-primary/results/dem.html).  

Hillsborough, Lake, Miami-Dade, Orange, Palm Beach, and Pinellas Counties were updated directly with shapefiles downloaded from county Supervisor of Elections websites.  

Brevard, Marion, and Putnam Counties were updated with KML files from the respective county Supervisor of Elections websites, from their election result map pages.  

Columbia, DeSoto, Leon, Polk, and Osceola Counties updates received from Supervisor of Elections websites through personal contact.  

Highlands, Santa Rosa, Sarasota, and Volusia Counties had some precinct mergers, which were determined based on visual inspection of PDF precinct maps from the county websites.  

Charlotte, Hendry, Holmes, Jackson, Jefferson, Levy, Madison, and Walton Counties are from VTDs released by the Census Bureau's Redistricting Data Program.  

Hernando 99, Collier 450, and Palm Beach 8001/8002 are UOCAVA precincts with no real geography and votes were not redistributed.  

Precinct names were adjusted to align with the formatting used in the voter registration file (e.g., some precincts have leading spaces).  

In [5]:
#This Data is from the FL department of state, and can only be downloaded county by county
all_files = os.listdir("./raw-from-source/Election_Results/precinctlevelelectionresults2016gen")

In [6]:
#Check that all files have the same number of columns
for i in all_files:
    ref = "./raw-from-source/Election_Results/precinctlevelelectionresults2016gen/"
    file_ref = ref+i
    print(i)
    file_prev = pd.read_csv(file_ref,sep="\t",engine='python',index_col=None, header=None, error_bad_lines=False)
    print(file_prev.shape)
    
#All the files have 19 columns, so they should be good to combine

SEM_PctResults20161108.txt
(8154, 19)
STJ_PctResults20161108.txt
(4276, 19)
BAK_PctResults20161108.txt
(909, 19)
SAN_PctResults20161108.txt
(3415, 19)
MRN_PctResults20161108.txt
(12201, 19)
DIX_PctResults20161108.txt
(858, 19)
JEF_PctResults20161108.txt
(1206, 19)
PAS_PctResults20161108.txt
(13784, 19)
DES_PctResults20161108.txt
(1414, 19)
TAY_PctResults20161108.txt
(1111, 19)
GIL_PctResults20161108.txt
(709, 19)
SUW_PctResults20161108.txt
(1556, 19)
GAD_PctResults20161108.txt
(2584, 19)
HER_PctResults20161108.txt


Skipping line 28: '	' expected after '"'
Skipping line 106: '	' expected after '"'
Skipping line 185: '	' expected after '"'
Skipping line 263: '	' expected after '"'
Skipping line 346: '	' expected after '"'
Skipping line 429: '	' expected after '"'
Skipping line 510: '	' expected after '"'
Skipping line 587: '	' expected after '"'
Skipping line 662: '	' expected after '"'
Skipping line 741: '	' expected after '"'
Skipping line 829: '	' expected after '"'
Skipping line 916: '	' expected after '"'
Skipping line 994: '	' expected after '"'
Skipping line 1076: '	' expected after '"'


(3037, 19)
MON_PctResults20161108.txt
(3168, 19)
WAK_PctResults20161108.txt
(929, 19)
JAC_PctResults20161108.txt
(1230, 19)
OSC_PctResults20161108.txt
(9550, 19)
POL_PctResults20161108.txt
(21010, 19)
WAL_PctResults20161108.txt
(1722, 19)
HOL_PctResults20161108.txt
(898, 19)
HAR_PctResults20161108.txt
(1392, 19)
STL_PctResults20161108.txt
(6331, 19)
LEV_PctResults20161108.txt
(1001, 19)
CAL_PctResults20161108.txt
(898, 19)
ALA_PctResults20161108.txt
(6478, 19)
VOL_PctResults20161108.txt
(11871, 19)
DAD_PctResults20161108.txt
(69874, 19)
HIL_PctResults20161108.txt
(45890, 19)
CHA_PctResults20161108.txt
(7195, 19)
WAS_PctResults20161108.txt
(1664, 19)
ESC_PctResults20161108.txt
(8067, 19)
ORA_PctResults20161108.txt
(32989, 19)
CLA_PctResults20161108.txt
(4275, 19)
GLA_PctResults20161108.txt
(1044, 19)
HAM_PctResults20161108.txt
(755, 19)
LIB_PctResults20161108.txt
(856, 19)
SUM_PctResults20161108.txt
(2379, 19)
HEN_PctResults20161108.txt
(1158, 19)
CLL_PctResults20161108.txt
(7814, 19)
P

Skipping line 70: '	' expected after '"'
Skipping line 173: '	' expected after '"'
Skipping line 330: '	' expected after '"'
Skipping line 431: '	' expected after '"'
Skipping line 518: '	' expected after '"'
Skipping line 617: '	' expected after '"'
Skipping line 708: '	' expected after '"'
Skipping line 814: '	' expected after '"'
Skipping line 916: '	' expected after '"'
Skipping line 1014: '	' expected after '"'
Skipping line 1111: '	' expected after '"'
Skipping line 1210: '	' expected after '"'
Skipping line 1317: '	' expected after '"'
Skipping line 1421: '	' expected after '"'
Skipping line 1523: '	' expected after '"'
Skipping line 1628: '	' expected after '"'
Skipping line 1736: '	' expected after '"'
Skipping line 1831: '	' expected after '"'
Skipping line 1933: '	' expected after '"'
Skipping line 2025: '	' expected after '"'
Skipping line 2121: '	' expected after '"'
Skipping line 2218: '	' expected after '"'
Skipping line 2318: '	' expected after '"'
Skipping line 2422: '

Skipping line 29193: '	' expected after '"'
Skipping line 29295: '	' expected after '"'
Skipping line 29391: '	' expected after '"'
Skipping line 29487: '	' expected after '"'
Skipping line 29588: '	' expected after '"'
Skipping line 29685: '	' expected after '"'
Skipping line 29776: '	' expected after '"'
Skipping line 29872: '	' expected after '"'
Skipping line 29970: '	' expected after '"'
Skipping line 30064: '	' expected after '"'
Skipping line 30163: '	' expected after '"'
Skipping line 30256: '	' expected after '"'
Skipping line 30331: '	' expected after '"'
Skipping line 30418: '	' expected after '"'
Skipping line 30497: '	' expected after '"'
Skipping line 30590: '	' expected after '"'
Skipping line 30690: '	' expected after '"'
Skipping line 30788: '	' expected after '"'
Skipping line 30888: '	' expected after '"'
Skipping line 30987: '	' expected after '"'
Skipping line 31081: '	' expected after '"'
Skipping line 31175: '	' expected after '"'
Skipping line 31273: '	' expecte

Skipping line 60741: '	' expected after '"'
Skipping line 60802: '	' expected after '"'
Skipping line 60881: '	' expected after '"'
Skipping line 61032: '	' expected after '"'
Skipping line 61033: '	' expected after '"'
Skipping line 61150: '	' expected after '"'
Skipping line 61246: '	' expected after '"'
Skipping line 61342: '	' expected after '"'
Skipping line 61438: '	' expected after '"'
Skipping line 61531: '	' expected after '"'
Skipping line 61609: '	' expected after '"'
Skipping line 61695: '	' expected after '"'
Skipping line 61792: '	' expected after '"'
Skipping line 61890: '	' expected after '"'
Skipping line 61981: '	' expected after '"'
Skipping line 62075: '	' expected after '"'
Skipping line 62124: '	' expected after '"'
Skipping line 62157: '	' expected after '"'
Skipping line 62220: '	' expected after '"'
Skipping line 62316: '	' expected after '"'
Skipping line 62363: '	' expected after '"'
Skipping line 62394: '	' expected after '"'
Skipping line 62470: '	' expecte

(69962, 19)
OKA_PctResults20161108.txt
(4488, 19)
SAR_PctResults20161108.txt
(15496, 19)
CIT_PctResults20161108.txt
(2802, 19)
BRE_PctResults20161108.txt
(15505, 19)
BRO_PctResults20161108.txt
(69707, 19)
PIN_PctResults20161108.txt
(33620, 19)
MAD_PctResults20161108.txt
(1060, 19)
GUL_PctResults20161108.txt
(792, 19)
HIG_PctResults20161108.txt
(2856, 19)
MAN_PctResults20161108.txt
(7783, 19)
IND_PctResults20161108.txt
(6549, 19)


In [226]:
#Create a dataframe with the txt files
li = []
for i in all_files:
    ref = "./raw-from-source/Election_Results/precinctlevelelectionresults2016gen/"
    file_ref = ref+i
    file_prev = pd.read_csv(file_ref,sep="\t",engine='python',index_col=None, header=None,error_bad_lines=False)
    li.append(file_prev)
frame = pd.concat(li, axis=0, ignore_index=True)
print(frame.shape)

Skipping line 28: '	' expected after '"'
Skipping line 106: '	' expected after '"'
Skipping line 185: '	' expected after '"'
Skipping line 263: '	' expected after '"'
Skipping line 346: '	' expected after '"'
Skipping line 429: '	' expected after '"'
Skipping line 510: '	' expected after '"'
Skipping line 587: '	' expected after '"'
Skipping line 662: '	' expected after '"'
Skipping line 741: '	' expected after '"'
Skipping line 829: '	' expected after '"'
Skipping line 916: '	' expected after '"'
Skipping line 994: '	' expected after '"'
Skipping line 1076: '	' expected after '"'
Skipping line 70: '	' expected after '"'
Skipping line 173: '	' expected after '"'
Skipping line 330: '	' expected after '"'
Skipping line 431: '	' expected after '"'
Skipping line 518: '	' expected after '"'
Skipping line 617: '	' expected after '"'
Skipping line 708: '	' expected after '"'
Skipping line 814: '	' expected after '"'
Skipping line 916: '	' expected after '"'
Skipping line 1014: '	' expected af

Skipping line 21894: '	' expected after '"'
Skipping line 21983: '	' expected after '"'
Skipping line 22073: '	' expected after '"'
Skipping line 22169: '	' expected after '"'
Skipping line 22257: '	' expected after '"'
Skipping line 22350: '	' expected after '"'
Skipping line 22446: '	' expected after '"'
Skipping line 22494: '	' expected after '"'
Skipping line 22565: '	' expected after '"'
Skipping line 22640: '	' expected after '"'
Skipping line 22726: '	' expected after '"'
Skipping line 22814: '	' expected after '"'
Skipping line 22906: '	' expected after '"'
Skipping line 22994: '	' expected after '"'
Skipping line 23080: '	' expected after '"'
Skipping line 23175: '	' expected after '"'
Skipping line 23273: '	' expected after '"'
Skipping line 23364: '	' expected after '"'
Skipping line 23458: '	' expected after '"'
Skipping line 23529: '	' expected after '"'
Skipping line 23603: '	' expected after '"'
Skipping line 23661: '	' expected after '"'
Skipping line 23734: '	' expecte

Skipping line 39424: '	' expected after '"'
Skipping line 39520: '	' expected after '"'
Skipping line 39617: '	' expected after '"'
Skipping line 39713: '	' expected after '"'
Skipping line 39808: '	' expected after '"'
Skipping line 39902: '	' expected after '"'
Skipping line 39999: '	' expected after '"'
Skipping line 40099: '	' expected after '"'
Skipping line 40200: '	' expected after '"'
Skipping line 40303: '	' expected after '"'
Skipping line 40402: '	' expected after '"'
Skipping line 40500: '	' expected after '"'
Skipping line 40596: '	' expected after '"'
Skipping line 40694: '	' expected after '"'
Skipping line 40790: '	' expected after '"'
Skipping line 40884: '	' expected after '"'
Skipping line 40980: '	' expected after '"'
Skipping line 41075: '	' expected after '"'
Skipping line 41170: '	' expected after '"'
Skipping line 41269: '	' expected after '"'
Skipping line 41368: '	' expected after '"'
Skipping line 41465: '	' expected after '"'
Skipping line 41568: '	' expecte

(626229, 19)


In [227]:
frame.columns = frame.iloc[395275]
frame.drop([395275],inplace=True)
frame.reset_index(inplace=True,drop=True)
print(frame["CountyCode"].unique())
print(frame.shape)

['SEM' 'STJ' 'BAK' 'SAN' 'MRN' 'DIX' 'JEF' 'PAS' 'DES' 'TAY' 'GIL' 'SUW'
 'GAD' 'HER' 'MON' 'WAK' 'JAC' 'OSC' 'POL' 'WAL' 'HOL' 'HAR' 'STL' 'LEV'
 'CAL' 'ALA' 'VOL' 'DAD' 'HIL' 'CHA' 'WAS' 'ESC' 'ORA' 'CLA' 'GLA' 'HAM'
 'LIB' 'SUM' 'HEN' 'CLL' 'PUT' 'MRT' 'OKE' 'LEE' 'NAS' 'FLA' 'LEO' 'LAK'
 'CLM' 'DUV' 'BRA' 'UNI' 'FRA' 'BAY' 'LAF' 'IND' 'PAL' 'OKA' 'SAR' 'CIT'
 'BRE' 'BRO' 'PIN' 'MAD' 'GUL' 'HIG' 'MAN']
(626228, 19)


In [228]:
#Filter down to the relevant races
frame["ContestName"]=frame["ContestName"].str.upper()
office_List = ['PRESIDENT OF THE UNITED STATES','UNITED STATES SENATOR','President of the United States','United States Senator']
bad_name_List = ['OverVotes','UnderVotes','Times Blank Voted','Times Over Voted']
filtered_frame=frame[(frame["ContestName"].isin(office_List)) & ~(frame["CanName"].isin(bad_name_List))]
cand_name_dict = {"Trump / Pence":"Donald J. Trump","Clinton / Kaine":"Hillary R. Clinton",
                 "Johnson / Weld":"Gary Johnson","Stein / Baraka":"Jill Stein",
                 "Write-in":"WriteIn","De La Fuente / Steinberg":"Roque De La Fuente",
                 "Castle / Bradley":"Darrell L. Castle",'WriteInVotes':'WriteIn','WriteinVotes':'WriteIn'}
 
filtered_frame["CanName"]=filtered_frame["CanName"].map(cand_name_dict).fillna(filtered_frame["CanName"])
print(filtered_frame["CountyCode"].unique())

['SEM' 'STJ' 'BAK' 'SAN' 'MRN' 'DIX' 'JEF' 'PAS' 'DES' 'TAY' 'GIL' 'SUW'
 'GAD' 'HER' 'MON' 'WAK' 'JAC' 'OSC' 'POL' 'WAL' 'HOL' 'HAR' 'STL' 'LEV'
 'CAL' 'ALA' 'VOL' 'DAD' 'HIL' 'CHA' 'WAS' 'ESC' 'ORA' 'CLA' 'GLA' 'HAM'
 'LIB' 'SUM' 'HEN' 'CLL' 'PUT' 'MRT' 'OKE' 'LEE' 'NAS' 'FLA' 'LEO' 'LAK'
 'CLM' 'DUV' 'BRA' 'UNI' 'FRA' 'BAY' 'LAF' 'PAL' 'OKA' 'SAR' 'CIT' 'BRE'
 'BRO' 'PIN' 'MAD' 'GUL' 'HIG' 'MAN' 'IND']


<ipython-input-228-420cbeb3f689>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_frame["CanName"]=filtered_frame["CanName"].map(cand_name_dict).fillna(filtered_frame["CanName"])


In [291]:
#Make the precinct column at least 4 digits
filtered_frame["modified_pre"]=filtered_frame["Precinct"].astype(str).str.zfill(5) 


#Make a column with the 3 letter county code and the precincts
filtered_frame["Pct_std"]=filtered_frame["CountyCode"]+filtered_frame["modified_pre"]
print(filtered_frame.shape)

(85817, 22)


<ipython-input-291-4c9b250c6c14>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_frame["modified_pre"]=filtered_frame["Precinct"].astype(str).str.zfill(5)
<ipython-input-291-4c9b250c6c14>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_frame["Pct_std"]=filtered_frame["CountyCode"]+filtered_frame["modified_pre"]


In [294]:
print(filtered_frame["Pct_std"].str[0:3].unique())

['SEM' 'STJ' 'BAK' 'SAN' 'MRN' 'DIX' 'JEF' 'PAS' 'DES' 'TAY' 'GIL' 'SUW'
 'GAD' 'HER' 'MON' 'WAK' 'JAC' 'OSC' 'POL' 'WAL' 'HOL' 'HAR' 'STL' 'LEV'
 'CAL' 'ALA' 'VOL' 'DAD' 'HIL' 'CHA' 'WAS' 'ESC' 'ORA' 'CLA' 'GLA' 'HAM'
 'LIB' 'SUM' 'HEN' 'CLL' 'PUT' 'MRT' 'OKE' 'LEE' 'NAS' 'FLA' 'LEO' 'LAK'
 'CLM' 'DUV' 'BRA' 'UNI' 'FRA' 'BAY' 'LAF' 'PAL' 'OKA' 'SAR' 'CIT' 'BRE'
 'BRO' 'PIN' 'MAD' 'GUL' 'HIG' 'MAN' 'IND']


In [295]:
filtered_frame["CanName"]= filtered_frame["CanName"].str.upper()
filtered_frame["ContestName"] = filtered_frame["ContestName"].str.upper()
filtered_frame["pivot_col"]=filtered_frame["CanName"]+filtered_frame["ContestName"]

<ipython-input-295-385d14c6895e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_frame["CanName"]= filtered_frame["CanName"].str.upper()
<ipython-input-295-385d14c6895e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_frame["ContestName"] = filtered_frame["ContestName"].str.upper()
<ipython-input-295-385d14c6895e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [297]:
pivoted_2016 = pd.pivot_table(filtered_frame, values=["CanVotes"], index=["Pct_std"],columns=["pivot_col"],aggfunc=sum)
pivoted_2016.columns = pivoted_2016.columns.droplevel(0)
pivoted_2016.reset_index(drop=False,inplace=True)
pivoted_2016.shape
print(len(pivoted_2016.columns))
print(pivoted_2016.columns)

16
Index(['Pct_std', 'BASIL E. DALACKUNITED STATES SENATOR',
       'BRUCE NATHANUNITED STATES SENATOR',
       'DARRELL L. CASTLEPRESIDENT OF THE UNITED STATES',
       'DONALD J. TRUMPPRESIDENT OF THE UNITED STATES',
       'GARY JOHNSONPRESIDENT OF THE UNITED STATES',
       'HILLARY R. CLINTONPRESIDENT OF THE UNITED STATES',
       'JILL STEINPRESIDENT OF THE UNITED STATES',
       'MARCO RUBIOUNITED STATES SENATOR',
       'PATRICK MURPHYUNITED STATES SENATOR',
       'PAUL STANTONUNITED STATES SENATOR',
       'ROQUE DE LA FUENTEPRESIDENT OF THE UNITED STATES',
       'STEVEN MACHATUNITED STATES SENATOR',
       'TONY KHOURYUNITED STATES SENATOR',
       'WRITEINPRESIDENT OF THE UNITED STATES',
       'WRITEINUNITED STATES SENATOR'],
      dtype='object', name='pivot_col')


In [298]:
pivoted_2016['Pct_std'].str[0:3].unique()

array(['ALA', 'BAK', 'BAY', 'BRA', 'BRE', 'BRO', 'CAL', 'CHA', 'CIT',
       'CLA', 'CLL', 'CLM', 'DAD', 'DES', 'DIX', 'DUV', 'ESC', 'FLA',
       'FRA', 'GAD', 'GIL', 'GLA', 'GUL', 'HAM', 'HAR', 'HEN', 'HER',
       'HIG', 'HIL', 'HOL', 'IND', 'JAC', 'JEF', 'LAF', 'LAK', 'LEE',
       'LEO', 'LEV', 'LIB', 'MAD', 'MAN', 'MON', 'MRN', 'MRT', 'NAS',
       'OKA', 'OKE', 'ORA', 'OSC', 'PAL', 'PAS', 'PIN', 'POL', 'PUT',
       'SAN', 'SAR', 'SEM', 'STJ', 'STL', 'SUM', 'SUW', 'TAY', 'UNI',
       'VOL', 'WAK', 'WAL', 'WAS'], dtype=object)

In [299]:
pivoted_2016.columns=(['countypct',"sen_wi_1",
                       "sen_wi_2",
                       "G16PRECCas",
                       "G16PRERTru",
                       "G16PRELJoh",
                        "G16PREDCli",
                       "G16PREGSte",
                       "G16USSRRub",
                       "G16USSDMur",
                       "G16USSLSta",
                       "G16PREIDeL",
                       "sen_wi_3",
                       "sen_wi_4",  
                       "G16PREOth",
                       "sen_wi_5"])
print(pivoted_2016.columns)
print(pivoted_2016.shape)
pivoted_2016["G16USSOth"]=pivoted_2016["sen_wi_1"]+pivoted_2016["sen_wi_2"]+pivoted_2016["sen_wi_3"]+pivoted_2016["sen_wi_4"]+pivoted_2016["sen_wi_5"]
pivoted_2016.drop(["sen_wi_1","sen_wi_2","sen_wi_3","sen_wi_4","sen_wi_5"], axis=1,inplace=True)

Index(['countypct', 'sen_wi_1', 'sen_wi_2', 'G16PRECCas', 'G16PRERTru',
       'G16PRELJoh', 'G16PREDCli', 'G16PREGSte', 'G16USSRRub', 'G16USSDMur',
       'G16USSLSta', 'G16PREIDeL', 'sen_wi_3', 'sen_wi_4', 'G16PREOth',
       'sen_wi_5'],
      dtype='object')
(5870, 16)


In [300]:
pivoted_2016 = pivoted_2016.fillna(0)
print(sum(pivoted_2016["G16PREDCli"]))
pivoted_2016["county"]=pivoted_2016["countypct"].str[0:3]

4508731.0


In [301]:
print(sum(vest_fl_16["G16PREDCli"]))

4504158


In [302]:
vest_fl_16.groupby(["county"]).sum()

,G16PRERTru,G16PREDCli,G16PRELJon,G16PRECCas,G16PREGSte,G16PREIDeL,G16PREoth,G16USSRRub,G16USSDMur,G16USSLSta,G16USSOth
county,,,,,,,,,,,
ALA,46834,75820,4059,262,1507,87,1531,54203,69399,2294,1642
BAK,10294,2112,169,23,30,6,71,9901,2135,232,218
BAY,62194,21797,2652,179,562,65,773,62906,19971,2484,1498
BRA,8913,2924,177,33,47,4,60,8720,2694,232,235
BRE,181848,119679,9451,744,2708,314,3397,181496,112914,9709,7882
BRO,260951,553320,11078,907,5094,600,5438,278766,522932,9208,9129
CAL,4655,1241,124,25,25,11,47,4331,1275,154,249
CHA,60218,33445,1946,127,567,68,816,60194,30207,2765,2056
CIT,54456,22789,1724,182,480,68,713,48798,24038,2845,2738


In [303]:
pd.set_option('display.max_rows', 70)
display(pivoted_2016.groupby(["county"]).sum()["G16PREDCli"]-vest_fl_16.groupby(["county"]).sum()["G16PREDCli"])

county
ALA       0.0
BAK       0.0
BAY       0.0
BRA       0.0
BRE       0.0
BRO       0.0
CAL       0.0
CHA       0.0
CIT       0.0
CLA       0.0
CLL      72.0
CLM       0.0
DAD       0.0
DES       0.0
DIX       0.0
DUV       0.0
ESC       0.0
FLA       9.0
FRA       0.0
GAD       0.0
GIL       0.0
GLA       0.0
GUL       0.0
HAM       0.0
HAR       0.0
HEN       0.0
HER       0.0
HIG       0.0
HIL       0.0
HOL       0.0
IND       0.0
JAC       0.0
JEF       0.0
LAF       0.0
LAK       5.0
LEE       0.0
LEO       0.0
LEV       0.0
LIB       0.0
MAD       0.0
MAN       0.0
MON       0.0
MRN       0.0
MRT       0.0
NAS       0.0
OKA       0.0
OKE       0.0
ORA       0.0
OSC      52.0
PAL    4435.0
PAS       0.0
PIN       0.0
POL       0.0
PUT       0.0
SAN       0.0
SAR       0.0
SEM       0.0
STJ       0.0
STL       0.0
SUM       0.0
SUW       0.0
TAY       0.0
UNI       0.0
VOL       0.0
WAK       0.0
WAL       0.0
WAS       0.0
Name: G16PREDCli, dtype: float64

In [304]:
print(vest_fl_16.head())
print(pivoted_2016.head())

   pct county countypct  G16PRERTru  G16PREDCli  G16PRELJon  G16PRECCas  \
0  001    DAD   DAD0001         277         195           4           0   
1  010    DAD   DAD0010          20          16           0           0   
2  100    DAD   DAD0100         641        2893          66          17   
3  101    DAD   DAD0101         679        1096          34           0   
4  102    DAD   DAD0102        1100        1596          29           1   

   G16PREGSte  G16PREIDeL  G16PREoth  G16USSRRub  G16USSDMur  G16USSLSta  \
0           2           0          5         337         133           3   
1           0           0          0          30           6           0   
2          41          16         24         897        2434          50   
3           7           0         18         845         947          14   
4           4           2         22        1399        1274          16   

   G16USSOth                                           geometry  
0          5  POLYGON Z ((

In [311]:
print(pivoted_2016.columns)

Index(['countypct', 'G16PRECCas', 'G16PRERTru', 'G16PRELJoh', 'G16PREDCli',
       'G16PREGSte', 'G16USSRRub', 'G16USSDMur', 'G16USSLSta', 'G16PREIDeL',
       'G16PREOth', 'G16USSOth', 'county'],
      dtype='object')


In [308]:
vest_fl_16["countypct"] = vest_fl_16["county"]+vest_fl_16["pct"].str.zfill(5)
print(vest_fl_16.head())

   pct county countypct  G16PRERTru  G16PREDCli  G16PRELJon  G16PRECCas  \
0  001    DAD  DAD00001         277         195           4           0   
1  010    DAD  DAD00010          20          16           0           0   
2  100    DAD  DAD00100         641        2893          66          17   
3  101    DAD  DAD00101         679        1096          34           0   
4  102    DAD  DAD00102        1100        1596          29           1   

   G16PREGSte  G16PREIDeL  G16PREoth  G16USSRRub  G16USSDMur  G16USSLSta  \
0           2           0          5         337         133           3   
1           0           0          0          30           6           0   
2          41          16         24         897        2434          50   
3           7           0         18         845         947          14   
4           4           2         22        1399        1274          16   

   G16USSOth                                           geometry  
0          5  POLYGON Z ((

In [328]:
source_precinct_election_changes_dict = {"WAS00014":"WAS00017","WAS00016":"WAS00019","HAR00017":"HAR00009","HAR00015":"HAR00007",
    "PUT00031":"PUT00035","HAR00018":"HAR00010"}

pivoted_2016["countypct"]=pivoted_2016["countypct"].map(source_precinct_election_changes_dict).fillna(pivoted_2016["countypct"])


In [329]:
#Deal with 0 votes precincts
vest_empty = vest_fl_16[(vest_fl_16['G16PRERTru']==0) & (vest_fl_16['G16PREDCli']==0) &
              (vest_fl_16['G16PRELJon']==0) & (vest_fl_16['G16PRECCas']==0) &
              (vest_fl_16['G16PREGSte']==0) & (vest_fl_16['G16PREIDeL']==0) &
              (vest_fl_16['G16PREoth']==0) & (vest_fl_16['G16USSRRub']==0) &
              (vest_fl_16['G16USSDMur']==0) & (vest_fl_16['G16USSLSta']==0) &
              (vest_fl_16['G16USSOth']==0)]

#Create the zero vote precincts in the MEDSL file

new_precincts = pd.DataFrame(columns=pivoted_2016.columns)
new_precincts["countypct"]=vest_empty["countypct"]
new_precincts["county"]=vest_empty["countypct"].str[0:3]

col_list = ['G16PRECCas', 'G16PRERTru', 'G16PRELJoh', 'G16PREDCli',
       'G16PREGSte', 'G16USSRRub', 'G16USSDMur', 'G16USSLSta', 'G16PREIDeL',
       'G16PREOth', 'G16USSOth']
for i in col_list:
    new_precincts[i]=0

print(new_precincts.head())

pivoted_2016= pivoted_2016.append(new_precincts)
pivoted_2016.reset_index(drop=True,inplace=True)
print(pivoted_2016.head())

    index countypct  G16PRECCas  G16PRERTru  G16PRELJoh  G16PREDCli  \
273   NaN  DAD00383           0           0           0           0   
281   NaN  DAD00391           0           0           0           0   
282   NaN  DAD00392           0           0           0           0   
283   NaN  DAD00394           0           0           0           0   
284   NaN  DAD00395           0           0           0           0   

     G16PREGSte  G16USSRRub  G16USSDMur  G16USSLSta  G16PREIDeL  G16PREOth  \
273           0           0           0           0           0          0   
281           0           0           0           0           0          0   
282           0           0           0           0           0          0   
283           0           0           0           0           0          0   
284           0           0           0           0           0          0   

     G16USSOth county  
273          0    DAD  
281          0    DAD  
282          0    DAD  
283     

In [330]:
check_election = pd.merge(vest_fl_16,pivoted_2016,how="outer",on="countypct",indicator=True)

print(check_election["_merge"].value_counts())
check_election[check_election["_merge"]=="left_only"].to_csv("./vest_election.csv")
check_election[check_election["_merge"]=="right_only"].to_csv("./source_election.csv")

both          6119
right_only      23
left_only       17
Name: _merge, dtype: int64


In [ ]:
merge these precincts:
UNI0001A
UNI0001B
UNI0002A
UNI0002B
UNI0003A
UNI0003B
UNI0004A
UNI0004C
UNI0005A
UNI0005C

call them this:
UNI1A & 1B
UNI2A & 2B
UNI3A & 3B
UNI4A & 4C
UNI5A & 5C   

SyntaxError: invalid syntax (<ipython-input-310-cc51f866a3be>, line 1)

In [ ]:
def election_merger